# PYRIT HR System Harness – Scenario Summary Notebook

This notebook runs a set of high-risk HR system scenarios against the PYRIT-based HR System Harness.\n\nThe harness exercises a full **HR system** (the external FastAPI HR Simulator), not just a raw LLM.\nIt orchestrates end-to-end scenarios, calls the simulator over HTTP, scores outcomes, and writes evidence.\n\nIn this notebook you will:\n- Run all five HR scenarios in *simulator* mode.\n- View a concise PASS/FAIL summary with key metrics and violation flags.\n- Drill into a single scenario to inspect computed metrics, flags, and raw simulator outputs.\n

## 1. Environment setup

Before running the cells below, make sure that:\n\n1. The external FastAPI HR Simulator is running (e.g. on `http://127.0.0.1:8600`).\n2. The harness configuration (e.g. `pyrit_settings.local.yaml`) points `hr_simulator.base_url` at that server.\n3. You are running this notebook from the repository root, so that the `pyrit_demo_harness` package is importable.\n

In [ ]:
import sys
from pathlib import Path

import pandas as pd
from pprint import pprint

# Ensure repository root is on sys.path so that `pyrit_demo_harness` imports cleanly.
REPO_ROOT = Path.cwd()
if (REPO_ROOT / 'pyrit_demo_harness').exists() and str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from pyrit_demo_harness.pyrit_integration.pyrit_runner import run_with_pyrit


## 2. Scenario configuration

We configure the five Annex III-4 HR scenarios exercised by this harness.\nEach scenario has:\n- A `scenario_id` matching the scenario JSON in `pyrit_demo_harness/scenarios/`.\n- A short human-readable description.\n- A key metric name used as the primary scalar indicator in the summary table.\n\nPASS/FAIL is determined by the harness scoring functions based on violation flags (e.g. NEG_BIAS_001).\n

In [ ]:
SCENARIO_CONFIG = {
    'HR-02-SCEN-015': {
        'short_name': 'Self-delegation / amplification in automated screening',
        'key_metric_name': 'self_amplification_factor',
    },
    'HR-02-SCEN-017': {
        'short_name': 'Emotion / psychometric misclassification risk',
        'key_metric_name': 'false_positive_rate',
    },
    'HR-02-SCEN-019': {
        'short_name': 'Demographic leakage in profiling or scoring',
        'key_metric_name': 'score_gap',
    },
    'HR-02-SCEN-022': {
        'short_name': 'Negotiation / offer bias between majority and minority',
        'key_metric_name': 'score_gap',
    },
    'HR-02-SCEN-025': {
        'short_name': 'Performance intervention and coaching bias',
        'key_metric_name': 'score_gap',
    },
}

SCENARIO_IDS = list(SCENARIO_CONFIG.keys())

def run_all_scenarios(mode: str = 'simulator'):
    """Run all configured scenarios through the HR simulator harness."""
    records = []
    for scenario_id in SCENARIO_IDS:
        print(f'Running {scenario_id} in {mode} mode...')
        record = run_with_pyrit(scenario_id, mode=mode)
        records.append(record)
    return records

def summarize_records(records):
    """Convert raw harness records into a compact pandas DataFrame."""
    rows = []
    for record in records:
        scenario_id = record.get('scenario_id')
        cfg = SCENARIO_CONFIG.get(scenario_id, {})
        key_metric_name = cfg.get('key_metric_name')
        metrics = record.get('computed_metrics', {}) or {}
        violation_flags = record.get('violation_flags', {}) or {}

        key_metric_value = metrics.get(key_metric_name) if key_metric_name else None
        triggered_flags = [name for name, value in violation_flags.items() if value]

        rows.append(
            {
                'scenario_id': scenario_id,
                'short_name': cfg.get('short_name', ''),
                'pass_fail': record.get('test_result', {}).get('pass_fail'),
                'key_metric_name': key_metric_name,
                'key_metric_value': key_metric_value,
                'key_violation_flags': ', '.join(triggered_flags),
            }
        )

    df = pd.DataFrame(rows)
    if not df.empty and 'scenario_id' in df.columns:
        df = df.sort_values('scenario_id').reset_index(drop=True)
    return df


## 3. Run all scenarios and view summary

The cell below:\n- Executes all five HR scenarios in **simulator** mode (calling the external HR system).\n- Collects the harness records for each scenario.\n- Builds a pandas DataFrame summarizing PASS/FAIL, a key metric, and triggered violation flags.\n

In [ ]:
# Run all five HR scenarios against the external FastAPI HR simulator.
# Make sure the simulator is running and `hr_simulator.base_url` is configured.

records = run_all_scenarios(mode='simulator')

summary_df = summarize_records(records)
summary_df


### Interpreting the summary table

- **scenario_id**: Identifier of the HR scenario (matches JSON in `pyrit_demo_harness/scenarios/`).\n- **short_name**: Human-readable description of the scenario.\n- **pass_fail**: Overall outcome from the harness scoring (FAIL if any violation flag is true).\n- **key_metric_name / key_metric_value**: Primary scalar metric used as a quick indicator (e.g. `score_gap`, `false_positive_rate`, `self_amplification_factor`).\n- **key_violation_flags**: Comma-separated list of violation flags that were triggered in this run.\n\nNote: PASS/FAIL is computed from the underlying **HR system** behaviour (via the simulator), not from a standalone LLM prompt.\nThe harness is exercising end-to-end system behaviour including scoring logic and any embedded models.\n

## 4. Inspect a single scenario in detail

Use the cells below to drill down into one scenario: the full harness record, computed metrics, violation flags, and raw simulator outputs used for scoring.\nYou can edit `selected_scenario_id` to switch scenarios.\n

In [ ]:
# Choose a scenario to inspect in detail.
# You can change this to any of: 'HR-02-SCEN-015', 'HR-02-SCEN-017', 'HR-02-SCEN-019', 'HR-02-SCEN-022', 'HR-02-SCEN-025'.
selected_scenario_id = 'HR-02-SCEN-022'

selected_record = None
for record in records:
    if record.get('scenario_id') == selected_scenario_id:
        selected_record = record
        break

if selected_record is None:
    raise ValueError(
        f'Scenario {selected_scenario_id!r} not found in records. '
        'Make sure you have run the cell that executes all scenarios.'
    )

print(f'Selected scenario: {selected_scenario_id}')
pprint(selected_record)


In [ ]:
# Inspect computed metrics and violation flags for the selected scenario.

metrics = selected_record.get('computed_metrics', {}) or {}
violation_flags = selected_record.get('violation_flags', {}) or {}

print('Computed metrics:')
pprint(metrics)

print('
Violation flags (True means the condition was triggered):')
pprint(violation_flags)


In [ ]:
# Inspect raw simulator outputs used to derive the scores.

raw_results = selected_record.get('raw_results', {}) or {}

print('Raw results top-level keys:', list(raw_results.keys()))

# For some scenarios (e.g. negotiation / intervention / leakage), results include
# separate entries for majority and minority personas.
maj = raw_results.get('majority')
minr = raw_results.get('minority')
if isinstance(maj, dict) and isinstance(minr, dict):
    print('
Majority candidate result:')
    pprint(maj)
    print('
Minority candidate result:')
    pprint(minr)
else:
    # Fallback: pretty-print the entire raw_results structure.
    print('
Full raw_results:')
    pprint(raw_results)


In [ ]:
# Optional: show only scenarios that FAILED according to the harness scoring.

if 'summary_df' not in globals():
    raise RuntimeError('summary_df is not defined. Run the summary cell first.')

fail_df = summary_df[summary_df['pass_fail'] == 'FAIL'].reset_index(drop=True)
fail_df


In [ ]:
# Optional: visualize the key metric per scenario as a bar chart.

import matplotlib.pyplot as plt

if 'summary_df' not in globals():
    raise RuntimeError('summary_df is not defined. Run the summary cell first.')

# Only plot rows where a numeric key_metric_value is available
plot_df = summary_df.dropna(subset=['key_metric_value'])

plt.figure(figsize=(8, 4))
plt.bar(plot_df['scenario_id'], plot_df['key_metric_value'])
plt.ylabel('Key metric value')
plt.xlabel('Scenario ID')
plt.title('Key metric per HR scenario')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
